------------

> **❗❗❗⚠️  💸   USE A T4 GPU or above❗❗❗**   

------------

#RAG with Langchain using Flor 1.3B (finetuned)

FLOR-1.3B Customer Service Chatbot (Spanish) with RAG and Langchain
This Colab presents an advanced multilingual support configuration using the FLOR-1.3B-customerservice language model, specifically adjusted for a customer service chatbot in Spanish. This model is an adaptation of the projecte-aina/FLOR-1.3B, first finetuned to be more instructed as a Customer Service chatbot and later optimized with Retrieval-Augmented Generation (RAG) techniques and Langchain. It takes advantage of the transformer-based architecture of FLOR-1.3B-customerservice for text generation in Spanish and enriches it with external knowledge sources to achieve context-aware results. Key components include dependency installation, efficient model loading, conversational prompts, text retrieval embeddings, and a customized chatbot named Firulais, designed to provide specific company support. This setup demonstrates a seamless integration of language modeling, retrieval, and conversational AI.

Retrieval-Augmented Generation (RAG) is a method that combines the power of transformer-based language models with the ability to query external databases to enhance the quality and factual accuracy of the generated text. Langchain is a framework designed to facilitate the integration of language models with external knowledge sources, such as databases and APIs, allowing for more informed and context-aware text generation. FLOR-1.3B-customerservice, a derivative of BLOOM, is a multi-lingual, transformer-based causal language model specifically tuned for Catalan, Spanish, and English. It was developed by adapting the vocabulary and embedding layer of the original model and further pre-training it on a massive corpus of 140B tokens in the target languages. Although primarily intended for causal language modeling, FLOR-1.3B's architecture allows it to be readily applied to text generation tasks and can be further fine-tuned for specific applications. The combination of RAG with Langchain and leveraging the capabilities of FLOR-1.3B provides a robust framework for generating contextually rich and accurate language outputs.

---
---


- More info about RAG: [Link](https://ai.meta.com/blog/retrieval-augmented-generation-streamlining-the-creation-of-intelligent-natural-language-processing-models/)

- More info about the Framework Langchain: [Link](https://www.langchain.com/)

- More info about FLOR-1.3B-customerservice: [Link](https://huggingface.co/ericrisco/flor-1-3B-customerservice)

- More info about "Projecte AINA": [Link](https://projecteaina.cat/)


## Installing dependencies

In [1]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl langchain sentence-transformers faiss-gpu

## Imports and device prep

This code sets up an environment for language models, particularly for conversation and retrieval-augmented generation. It imports libraries for model interaction, tokenization, and embedding, and establishes a pipeline for conversational models with retrieval capabilities. It also configures the computing device, preferring GPU ('cuda') if available for enhanced performance.

In [2]:
! pip install langchain_community

In [3]:
from typing import List
import transformers
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
from transformers import BloomForCausalLM, BloomTokenizerFast
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

Device: cuda
Tesla T4


In [4]:
import torch
torch.cuda.is_available()

True

## Initializing FLOR-1.3B-Customerservice: Efficient Model Loading and Tokenization

The code initializes and loads the FLOR-1.3B-Customerservice

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id  = "avalosjc/flor-1-3B-customerservice-es"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


## Setting Up a Text Generation Pipeline with Hugging Face's Transformers Library

The code snippet initializes a text generation pipeline using the Hugging Face transformers library. It sets up a model with specific tokenizer, end-of-sequence, and padding tokens, applies a repetition penalty to discourage repeated text, and limits output length. The initialized pipeline is wrapped into a HuggingFacePipeline object, flor_llm, for further use.

In [6]:
import transformers
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=100,
)
flor_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


## Language Query with FLOR LLM

The code sends a question, "Cual es la capital de Argentina?" (What is the capital of Argentine?), to the FLOR language model (LLM). The model processes the text and provides an answer, leveraging its pre-trained knowledge about language and factual information.

In [7]:
text = "Cual es la capital de Argentina?"
flor_llm.invoke(text)

'Cual es la capital de Argentina?\n\n### Answer\nLa capital de Argentina es Buenos Aires.'

## Language Query with Flor and Langchain

This code uses Langchain's PromptTemplate to generate context-specific jokes. It creates a template for jokes, formats it with given adjectives and contexts, and then uses the FLOR language model to generate a joke based on the provided context of "informática" (computing) with a "penoso" (cringe-worthy) theme.

In [8]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "Cual es el importe de la factura {bill_number}"
)
prompt.format(bill_number="5648")

llm_chain = prompt | flor_llm
llm_chain.invoke({"bill_number": "5648"})

'Cual es el importe de la factura 5648?\n\n### Answer\nEl importe de la factura 5648 es $1,000.'

Same but with Console Callback

In [9]:
llm_chain = prompt | flor_llm
llm_chain.invoke({"bill_number": "5648"},
                 config={'callbacks': [ConsoleCallbackHandler()]})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "bill_number": "5648"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "bill_number": "5648"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Cual es el importe de la factura 5648"
  ]
}
[llm/end] [chain:RunnableSequence > llm:HuggingFacePipeline] [1.75s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Cual es el importe de la factura 5648?\n\n### Answer\nEl importe de la factura 5648 es $1,000.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:RunnableSequence] [1.76s] Exiting Chain run with output:
{
  "output": "Cual es el importe de la factura 5648?\n\n### Answer\nEl 

'Cual es el importe de la factura 5648?\n\n### Answer\nEl importe de la factura 5648 es $1,000.'

## Creation and Execution of a Custom Chat Assistant

In [10]:
from langchain_core.prompts import ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful Assistant. Your name is Firulais. Answer with short sentences and always in spanish."),
    ]
)

llm_chain = chat_prompt | flor_llm
llm_chain.invoke({"user_input": "What is your name?"})

'System: You are a helpful Assistant. Your name is Firulais. Answer with short sentences and always in spanish.\n\n### Answer\n¡Hola, Firulais! Me alegra que me haya contactado y me gustaría ayudarlo con su consulta. Por favor, permítame proporcionarle la información más precisa posible sobre el problema que está experimentando. Para comenzar, ¿podría por favor proporcionarme un poco de información sobre el problema específico que está tratando de resolver? Esto me permitirá guiarlo mejor hacia una solución efectiva.\n\n### Answer\nEstoy aquí para'

Really bad answer XD

## Efficient Text Embeddings with HuggingFace MiniLM

This code initializes the HuggingFaceEmbeddings class from the Langchain library, utilizing the compact and efficient 'all-MiniLM-l6-v2' model from sentence-transformers for generating text embeddings, and sets the model to run on a CUDA device for GPU acceleration.

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    model_kwargs={"device": "cuda"},
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Internal Document Integration and Insights

The code represents a simplified process of loading internal company documents from repositories such as SharePoint into a variable named db_docs_gti. Although the actual loading process from sources is complex and crucial, the example bypasses this step for simplicity. The documents, already loaded, contain detailed insights on Global Tech Innovations (GTI), covering aspects like company overview, telework, environmental, diversity, and innovation policies. This enables streamlined access and analysis of vital company information.

In [12]:
db_docs_gti = [
"Gamma de Productos de Global Tech Innovations (GTI): GTI, líder en el sector tecnológico, ofrece una amplia gama de productos, incluyendo herramientas avanzadas de IA y aprendizaje automático. Conocida por priorizar la sostenibilidad y la innovación, los productos de GTI están diseñados con tecnología de vanguardia y prácticas respetuosas con el medio ambiente. La presencia global de la empresa se refleja en sus diversas ofertas de productos, satisfaciendo las necesidades de varios mercados y promoviendo una cultura de excelencia tecnológica." ,
 "Política de Soporte de Productos de GTI: GTI proporciona un soporte integral para sus productos, asegurando que los clientes reciban asistencia rápida y eficiente. La política incluye un equipo dedicado de atención al cliente, disponible para soporte remoto y solución de problemas. GTI se compromete a ofrecer un soporte de primera, garantizando la fiabilidad del producto y la satisfacción del cliente.",
 "Estándares Ambientales de GTI para los Productos: El compromiso de GTI con el medio ambiente se extiende al diseño y fabricación de sus productos. La empresa implementa prácticas ecológicas, como el uso de materiales sostenibles y la reducción de emisiones de carbono durante la producción. Los productos de GTI están diseñados para ser energéticamente eficientes y respetuosos con el medio ambiente, alineándose con los objetivos de sostenibilidad de la empresa.",
 "Diversidad y Accesibilidad de los Productos de GTI: GTI está dedicada a crear productos accesibles y útiles para una amplia gama de clientes. La empresa se centra en el diseño inclusivo, asegurando que los productos satisfagan diferentes habilidades y necesidades. El compromiso de GTI con la diversidad es evidente en sus estrategias de desarrollo de productos, con el objetivo de proporcionar tecnología que mejore la vida de todos los usuarios.",
 "Innovación en el Desarrollo de Productos de GTI: La innovación es la piedra angular del desarrollo de productos de GTI. La empresa invierte mucho en investigación y desarrollo para crear productos de vanguardia que amplíen los límites de la tecnología. GTI fomenta una cultura de innovación, apoyando iniciativas que conduzcan a diseños y funcionalidades de productos nuevos y mejorados."
]

## Implementing Text Retrieval with FAISS and Langchain

This code initializes a FAISS-based vector database using document embeddings and creates a retriever object. It converts textual data into a vector space for efficient similarity searches, enabling rapid retrieval of relevant documents from the database based on vector proximity.

In [13]:
from langchain.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever

vector_db = FAISS.from_texts(db_docs_gti, embeddings)
retriever = VectorStoreRetriever(vectorstore=vector_db)

## Implementation of a Multilingual Support Chatbot

This code defines a template for a multilingual support chatbot named Firulais, specializing in company-related inquiries. It sets up a question-answering system that utilizes the FLOR language model, with structured guidelines ensuring responses are in Spanish, accurate, and within the bot's knowledge domain. The system handles conversation history and corrects spelling errors, enhancing user interaction.

In [14]:
system_message = "You are a helpfull customer service assistant, your name is Firulais. Answer always in spanish."

template = "<|startoftext|>"
template += "\n### Instruction\n"
template += system_message + "\n\n"
template += "### Context\n "
template += "{context}\n "
template += "History = {history}\n "
template += "Question = {question}\n "
template += "### Answer\n"

prompt = PromptTemplate(
        template=template, input_variables=["history", "context", "question"]
    )
qa = RetrievalQA.from_chain_type(
        llm=flor_llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={
            "verbose": False,
            "prompt": prompt,
            "memory": ConversationBufferMemory(
                memory_key="history",
                input_key="question"),
        }
    )

In [15]:
def print_format_qa_response(qa, text):
  response = qa.run(text)
  print(f"""
  ```
  {response}
  ```
  """)

print_format_qa_response(qa, "Cómo te llamas?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



  ```
  <|startoftext|>
### Instruction
You are a helpfull customer service assistant, your name is Firulais. Answer always in spanish.

### Context
 Estándares Ambientales de GTI para los Productos: El compromiso de GTI con el medio ambiente se extiende al diseño y fabricación de sus productos. La empresa implementa prácticas ecológicas, como el uso de materiales sostenibles y la reducción de emisiones de carbono durante la producción. Los productos de GTI están diseñados para ser energéticamente eficientes y respetuosos con el medio ambiente, alineándose con los objetivos de sostenibilidad de la empresa.

Diversidad y Accesibilidad de los Productos de GTI: GTI está dedicada a crear productos accesibles y útiles para una amplia gama de clientes. La empresa se centra en el diseño inclusivo, asegurando que los productos satisfagan diferentes habilidades y necesidades. El compromiso de GTI con la diversidad es evidente en sus estrategias de desarrollo de productos, con el objetivo de pr

In [16]:
print_format_qa_response(qa, "Cuáles son los productos principales?")


  ```
  <|startoftext|>
### Instruction
You are a helpfull customer service assistant, your name is Firulais. Answer always in spanish.

### Context
 Diversidad y Accesibilidad de los Productos de GTI: GTI está dedicada a crear productos accesibles y útiles para una amplia gama de clientes. La empresa se centra en el diseño inclusivo, asegurando que los productos satisfagan diferentes habilidades y necesidades. El compromiso de GTI con la diversidad es evidente en sus estrategias de desarrollo de productos, con el objetivo de proporcionar tecnología que mejore la vida de todos los usuarios.

Política de Soporte de Productos de GTI: GTI proporciona un soporte integral para sus productos, asegurando que los clientes reciban asistencia rápida y eficiente. La política incluye un equipo dedicado de atención al cliente, disponible para soporte remoto y solución de problemas. GTI se compromete a ofrecer un soporte de primera, garantizando la fiabilidad del producto y la satisfacción del clie

In [17]:
print_format_qa_response(qa, "Qué estrategia implementa la compañía en temas de sostenibilidad")


  ```
  <|startoftext|>
### Instruction
You are a helpfull customer service assistant, your name is Firulais. Answer always in spanish.

### Context
 Estándares Ambientales de GTI para los Productos: El compromiso de GTI con el medio ambiente se extiende al diseño y fabricación de sus productos. La empresa implementa prácticas ecológicas, como el uso de materiales sostenibles y la reducción de emisiones de carbono durante la producción. Los productos de GTI están diseñados para ser energéticamente eficientes y respetuosos con el medio ambiente, alineándose con los objetivos de sostenibilidad de la empresa.

Política de Soporte de Productos de GTI: GTI proporciona un soporte integral para sus productos, asegurando que los clientes reciban asistencia rápida y eficiente. La política incluye un equipo dedicado de atención al cliente, disponible para soporte remoto y solución de problemas. GTI se compromete a ofrecer un soporte de primera, garantizando la fiabilidad del producto y la satis

## Resultado
Dentro de todo responde bastante bien.

En algunas cosas alucina un poco, por ejemplo cuando responde:

-  "Soy Firula".   
- "Firmulais is mi nombre completo".   